# Package Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import random
import pandas as pd
import numpy as np
print('Done importing packages')

Done importing packages


# Read in Property List

In [2]:
# Reading property list
df_nexu = pd.read_excel('C:/Users/Longfei/Desktop/For Work/PropertyValueCrawler/zillmere_full.xlsx',
                        sheet_name='zillmere_full')

# creat the search_address column
df_nexu['search_addr'] = df_nexu['streetaddress']+' '+\
                        df_nexu['locality']+' '+df_nexu['state']+' '+\
                        df_nexu['postcode'].astype(str)

# get the list of addresses that have missing built year
df_valid = df_nexu[df_nexu['yearbuilt'].notnull()]
df_missing = df_nexu[df_nexu['yearbuilt'].isna()]
# df_nexu.columns.tolist()

In [3]:
# get the unique addresses into a tmp dataframe
df_tmp = pd.DataFrame()
df_tmp['search_addr'] = df_missing['search_addr'].unique().tolist()
df_tmp.dropna(inplace=True)
df_tmp['yearbuilt'] = np.nan
df_tmp

,search_addr,yearbuilt
0,5 Midyim Street Zillmere QLD 4034,NaN
1,23/49 Handford Road Zillmere QLD 4034,NaN
2,4/51 Glasgow Street Zillmere QLD 4034,NaN
3,26 Ridgmont Street Zillmere QLD 4034,NaN
4,1/51 Glasgow Street Zillmere QLD 4034,NaN
...,...,...
2864,Unit 9/22 Grasspan Street Zillmere QLD 4034,NaN
2865,2/402 Zillmere Rd Zillmere QLD 4034,NaN
2866,5/3 Tabill Street Zillmere QLD 4034,NaN
2867,3/49 Rodway St Zillmere QLD 4034,NaN


# Function Definitions

In [4]:
# url = 'https://www.propertyvalue.com.au/'
# driver = webdriver.Edge()

In [5]:
# driver.get(url)

In [6]:
# Function Definition -- Initialise page
def new_page():
  # Crawler Initiation -- Opening the browser of choice
  url = 'https://www.propertyvalue.com.au/'
  driver = webdriver.Edge()
  driver.get(url)
  driver.implicitly_wait(random.randint(2, 5))
#   print(driver.title)

  # Accepting Cookies
  elements = driver.find_elements(By.XPATH, "//button[@id='acceptCookieButton']")
  if len(elements)>0:
    elements[0].click()
  else:
    pass
  return driver

In [7]:
# Function Definition -- Address Search
def search_for_yearbuilt(driver,address):
  print('Processing addr -- ',address,'...')
  
#   # verify if on the landing page
#   while (EC.presence_of_element_located((By.ID, "data-rismark")) is not True):
#     elements = driver.find_elements(By.XPATH, "//div[@class='logo property-value']")
#     elements[0].click()
#     driver.implicitly_wait(random.randint(2, 5))
#   print('    Landing page located...')
  # input address
#   address = "1516/3 Yarra Street South Yarra Vic 3141"
  step_one = driver.find_elements(By.XPATH, "//input[@id='propertysearch']")
  step_one[0].send_keys(address)
  driver.implicitly_wait(random.randint(2, 5))
  
  # click the first suggestion
  try:
    step_two = driver.find_elements(By.XPATH, "//li[@class='jquery-autocomplete-selected-item acSelect']")
    step_two[0].click()
  except:
    print('    SKIP, Address recommendation not responding.')
    return np.nan
  driver.implicitly_wait(random.randint(5,10))

  # Read the property address and check with the input
  elements = driver.find_elements(By.XPATH, "//span[@class='streetAddress']")
  if len(elements)<1:
    print('    SKIP, Ambiguous search address')
    print()
    return np.nan
  street_addr = elements[0].text

#   elements = driver.find_elements(By.XPATH, "//span[@itemprop='addressLocality']")
#   locality = elements[0].text

#   elements = driver.find_elements(By.XPATH, "//span[@itemprop='addressRegion']")
#   state = elements[0].text

#   elements = driver.find_elements(By.XPATH, "//span[@itemprop='postalCode']")
#   postcode = elements[0].text

#   return_addr = (street_addr.lower()+locality.lower()+state.lower()+postcode.lower()).replace(' ','')

#   # assert if the addresses match
#   if return_addr != address.lower().replace(' ',''):
#     print('    SKIP, Mismatching addresses')
#     print('    search_addr:', address)
#     print('    return_addr:', street_addr+' '+locality+' '+state+' '+postcode)
#     print()
#     return np.nan

  # get the property table
  elements = driver.find_elements(By.XPATH, "//table[@class='property-table']")
  
  # parse the table for yearbuilt
  this_yearbuilt = ''
  for elm in elements:
    results = elm.text.splitlines()
    for res in results:
      if 'Year Built' in res:
        try:this_yearbuilt = int(res.replace('Year Built','').strip())
        except:this_yearbuild = np.nan
        break
    if this_yearbuilt != '':
      break
  
#   # reset the page
#   driver.quit()
#   elements = driver.find_elements(By.XPATH, "//div[@class='logo property-value']")
#   elements[0].click()
#   driver.implicitly_wait(random.randint(2, 5))
  print('    DONE. --- Year Built =',this_yearbuilt)
  print()
  
  # return
  if this_yearbuilt == '':
    return np.nan
  else:
    return this_yearbuilt

# Main Loop -- Crawling the Year Built

In [8]:
# Loading a new page
url = 'https://www.propertyvalue.com.au/'
driver = new_page()

In [9]:
# Calling the crawler to obtain yearbuilt for all properties
year_built_dict = {}
for _,listing in df_tmp.iterrows():
  
  # find the year built for the current address
  year_built_dict[listing['search_addr']] = search_for_yearbuilt(driver,listing['search_addr'])
   
  # force a sleep
  sleep_time = round(random.uniform(1.1,3.2),1)
  time.sleep(sleep_time)

  # reload the page
  driver.get(url)

Processing addr --  5 Midyim Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 2018

Processing addr --  23/49 Handford Road Zillmere QLD 4034 ...
    DONE. --- Year Built = 1990

Processing addr --  4/51 Glasgow Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 2007

Processing addr --  26 Ridgmont Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 1950

Processing addr --  1/51 Glasgow Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 2008

Processing addr --  1 Harold Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 1960

Processing addr --  5/20 Pioneer Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 2010

Processing addr --  1/5 Rodway Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 2011

Processing addr --  1/84 Hirschfield Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 2011

Processing addr --  9/67 Rodway Street Zillmere QLD 4034 ...
    DONE. --- Year Built = 2016

Processing addr --  17 Fahey Street Zillmere QLD 4034 ...


In [10]:
# year_built_dict

In [11]:
df_tmp2 = pd.DataFrame().from_dict(year_built_dict,orient='index')
df_tmp2['search_addr'] = df_tmp2.index
df_tmp2.rename(columns={0:'yearbuilt'},inplace=True)
df_tmp2.reset_index(drop=True, inplace=True)
df_tmp2

,yearbuilt,search_addr
0,2018.0,5 Midyim Street Zillmere QLD 4034
1,1990.0,23/49 Handford Road Zillmere QLD 4034
2,2007.0,4/51 Glasgow Street Zillmere QLD 4034
3,1950.0,26 Ridgmont Street Zillmere QLD 4034
4,2008.0,1/51 Glasgow Street Zillmere QLD 4034
...,...,...
2863,2006.0,Unit 9/22 Grasspan Street Zillmere QLD 4034
2864,1994.0,2/402 Zillmere Rd Zillmere QLD 4034
2865,NaN,5/3 Tabill Street Zillmere QLD 4034
2866,2012.0,3/49 Rodway St Zillmere QLD 4034


In [12]:
# Merge tmp2 back into missing
df_missing_tmp = df_missing.drop(columns=['yearbuilt'])
df_missing2 = pd.merge(df_missing_tmp,df_tmp2,how='left',on='search_addr')
df_missing2['yearbuilt'].value_counts(dropna=False)

NaN       1845
1950.0     356
1960.0     292
2015.0     252
2010.0     220
          ... 
1948.0       1
1866.0       1
1969.0       1
1910.0       1
1900.0       1
Name: yearbuilt, Length: 75, dtype: int64

In [13]:
# concatenate missing and non missing
df_output = pd.concat([df_valid,df_missing2],verify_integrity=True,ignore_index=True)
df_output

,listingid,reaid,projectreaid,propertyreaid,portalurl,headline,description_trunc,streetaddress,locality,postcode,...,avmestimate_value,avmestimate_lastupdated,avmestimate_confidence,avmestimate_min,avmestimate_max,markettrends_annualgrowth,markettrends_mediansoldprice,floorarea,landarea.1,search_addr
0,18515331,137648282,NaN,NaN,https://www.realestate.com.au/sold/property-ho...,Attention Developers & Savvy Investors!,Invest in this highly sought-after 1280m2 site...,283 Zillmere Road,Zillmere,4034,...,847970.0,2020-05-04,high,800000.0,900000.0,0.024,462500.0,NaN,643.0,283 Zillmere Road Zillmere QLD 4034
1,19271373,138005198,NaN,NaN,https://www.realestate.com.au/property-house-q...,Great family home on 548m2 block!,This spacious low set 4 bedroom home is ideall...,12 Fisher Parade,Zillmere,4034,...,658968.0,2020-05-03,high,575000.0,750000.0,0.033,540388.0,NaN,519.0,12 Fisher Parade Zillmere QLD 4034
2,18948787,137856790,NaN,NaN,https://www.realestate.com.au/property-house-q...,Refurbished Postwar on 764sqm - Walk to Train ...,Perfectly positioned with a north/south aspect...,98 Crowley Street,Zillmere,4034,...,501946.0,2020-05-03,high,475000.0,550000.0,0.023,460000.0,NaN,675.0,98 Crowley Street Zillmere QLD 4034
3,19101035,137927650,NaN,NaN,https://www.realestate.com.au/property-house-q...,High-Set Family Home with Dual Living!,"Set in a great location central to everything,...",17 Murphy Road,Zillmere,4034,...,453450.0,2020-05-04,high,400000.0,500000.0,0.023,460000.0,NaN,380.0,17 Murphy Road Zillmere QLD 4034
4,19110658,137935606,NaN,NaN,https://www.realestate.com.au/property-house-q...,Fully renovated Federation era beauty est. 1903,"Located 250m to the Aspley border, this meticu...",39 Weston Street,Zillmere,4034,...,602364.0,2020-05-04,high,525000.0,675000.0,0.024,462500.0,NaN,628.0,39 Weston Street Zillmere QLD 4034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647,5685819,420164518,NaN,NaN,http://www.realestate.com.au/property-townhous...,THREE BEDROOM TOWNHOUSE - SORRY NO PETS !,This lovely two storey townhouse is one to sna...,16/23 Pretoria Street,Zillmere,4034,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16/23 Pretoria Street Zillmere QLD 4034
6648,6027775,420781430,NaN,NaN,http://www.realestate.com.au/property-apartmen...,Generous Fully Air-Conditioned Two Bedroom Unit,Located on the first floor this spacious two b...,5/3 Tabill Street,Zillmere,4034,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5/3 Tabill Street Zillmere QLD 4034
6649,1440513,115038215,NaN,NaN,http://www.realestate.com.au/property-unit-qld...,Looking to get in the market ?,Stunning Units for sale just about to be compl...,NaN,Zillmere,4034,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6650,4217745,417523118,NaN,NaN,http://www.realestate.com.au/property-unit-qld...,Two Bedroom Townhouse with Private Courtyard!,This spacious 2 bedroom townhouse features que...,3/49 Rodway St,Zillmere,4034,...,320427.0,2020-05-16,high,300000.0,350000.0,-0.024,296511.0,NaN,80.0,3/49 Rodway St Zillmere QLD 4034


In [14]:
# output
df_output.to_excel('C:/Users/Longfei/Desktop/For Work/PropertyValueCrawler/zillmere_full_enriched.xlsx',
                   index=False)